# 1. 강의영상 

<https://youtu.be/playlist?list=PLQqh36zP38-y6dmKt32J5hobALnT8wigT&si=ScK7ryQpemMS4cJd>

# 2. Imports 

In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import sklearn.linear_model 
import sklearn.tree
import sklearn.model_selection

# 3. Data 

In [3]:
# n = 10000
# Supplement = np.random.choice([True, False], n)
# Exercise = np.random.choice([False, True], n)
# Weight_Loss = np.where(
#     (~Supplement & (~Exercise)),
#     np.random.normal(loc=0, scale=1, size=n),  
#     np.where(
#         (Supplement & (Exercise)),
#         np.random.normal(loc=15.00, scale=1, size=n),
#         np.where(
#             (~Supplement & (Exercise)),
#             np.random.normal(loc=5.00, scale=1, size=n),
#             np.random.normal(loc=0.5, scale=1, size=n)
#         )
#     )
# )
# df = pd.DataFrame({
#     'Supplement': Supplement,
#     'Exercise': Exercise,
#     'Weight_Loss': Weight_Loss
# })
df_train = pd.read_csv('https://raw.githubusercontent.com/guebin/MP2023/main/posts/weightloss.csv')

In [4]:
df_train

Supplement  Exercise  Weight_Loss
0          False     False    -0.877103
1           True     False     1.604542
2           True      True    13.824148
3           True      True    13.004505
4           True      True    13.701128
...          ...       ...          ...
9995        True     False     1.558841
9996       False     False    -0.217816
9997       False      True     4.072701
9998        True     False    -0.253796
9999       False     False    -1.399092

[10000 rows x 3 columns]

In [5]:
df_train.pivot_table(index='Supplement',columns='Exercise',values='Weight_Loss')

Exercise       False      True 
Supplement                     
False       0.021673   4.991314
True        0.497573  14.966363

`-` 운동과 체중감량보조제를 병행하면 시너지가 나는 것 같음 

- 운동만 하면 --> 5kg빠짐
- 약만 먹으면 -->  0.5정도 빠짐
- 운동 + 약 --> 15kg정도 빠짐

# 4. 분석

`-` 분석1: 모형을 아래와 같이 본다. -- 언더피팅

- ${\bf X}$: `Supplement`, `Exercise`
- ${\bf y}$: `Weight_Loss`

In [8]:
# step1
X = df_train[['Supplement','Exercise']]
y = df_train['Weight_Loss']
# step2 
predictr = sklearn.linear_model.LinearRegression()
# step3
predictr.fit(X,y)
# step4 
df_train['Weight_Loss_hat'] = predictr.predict(X)
#---#
print(f'train score = {predictr.score(X,y):.4f}')

train score = 0.8208


In [10]:
df_train.pivot_table(index='Supplement',columns='Exercise',values='Weight_Loss')

Exercise       False      True 
Supplement                     
False       0.021673   4.991314
True        0.497573  14.966363

In [11]:
df_train.pivot_table(index='Supplement',columns='Exercise',values='Weight_Loss_hat')

Exercise       False      True 
Supplement                     
False      -2.373106   7.374557
True        2.845934  12.593598

- 운동을 하면 10키로 감량효과가 있다고 추정하고 있음. 
- 보충제를 먹으면 5키로 감량효과가 있다고 추정하고 있음. 
- 대충 (10,5)의 숫자를 바꿔가면서 적합해봤는데 이게 최선이라는 의미임 

In [13]:
df_train.head()

Supplement  Exercise  Weight_Loss  Weight_Loss_hat
0       False     False    -0.877103        -2.373106
1        True     False     1.604542         2.845934
2        True      True    13.824148        12.593598
3        True      True    13.004505        12.593598
4        True      True    13.701128        12.593598

`-` 추가효과가 있는지 조사.

|index|약의효과|운동의효과|추가효과|
|:-----:|:----------:|:-------:|:--------:|
|$0$|$0$|$0$|$0$|
|$1$|$1$|$0$|$0$|
|$2$|$1$|$1$|$1$|
|$3$|$1$|$1$|$1$|

`-` 분석2: 모형을 아래와 같이 본다. -- 딱 맞아요

- ${\bf X}$: `Supplement`, `Exercise`, `Supplement` $\times$ `Exercise`
- ${\bf y}$: `Weight_Loss`

> Note: 기본적인 운동의 효과 및 보조제의 효과는 각각 `Supplement`, `Exercise` 로 적합하고 운동과 보조제의 시너지는 `Supplement`$\times$`Exercise` 로 적합한다. 

In [14]:
# step1 
X = df_train.eval('Interaction = Supplement * Exercise')[['Supplement','Exercise','Interaction']]
y = df_train['Weight_Loss']
# step2 
predictr = sklearn.linear_model.LinearRegression()
# step3 
predictr.fit(X,y)
# step4 -- pass 
df_train['Weight_Loss_hat'] = predictr.predict(X)
#---#
print(f'train score = {predictr.score(X,y):.4f}')

train score = 0.9728


In [19]:
df_train.pivot_table(index='Supplement',columns='Exercise',values='Weight_Loss')

Exercise       False      True 
Supplement                     
False       0.021673   4.991314
True        0.497573  14.966363

In [20]:
df_train.pivot_table(index='Supplement',columns='Exercise',values='Weight_Loss_hat')

Exercise       False      True 
Supplement                     
False       0.021673   4.991314
True        0.497573  14.966363

- 운동의 효과는 5정도 감량효과가 있다고 추정함. 
- 보충제를 먹으면 0.5키로 감량효과가 있다고 추정함.
- 다만 운동을 하면서 보충제를 함께 먹을 경우 발생하는 추가적인 시너지효과가 9.5정도라고 추정하는 것임. 